In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pefile
import sqlite3
from dataLoader_Wrapper import *
from BinaryClassifier import *
import random
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [87]:
dataThing = da.DataAcquirer('H:/Datasets/Bigger_dataset/data.sqlite', 'H:/Datasets/Bigger_dataset/dataset/')
dataThing.bootup()
def dataGetter(dataThing, numChunks=100, chunkLength=1000):
    num_chunks = numChunks
    chunk_length = chunkLength
    x86rows = dataThing.filter('binaries', 'platform', 'x86')
    x64rows = dataThing.filter('binaries', 'platform', 'x64')
    print("Getting file info")
    x86file_info = dataThing.getFileInfo(x86rows)
    x64file_info = dataThing.getFileInfo(x64rows)
    print("Got file info")
    #     randomly select n files from the list
    x86chunkList = extractChunks(x86file_info, chunk_length, num_chunks)
    x64chunkList = extractChunks(x64file_info, chunk_length, num_chunks)
    print("Extracted chunks")
    print(f'x86 chunks: {len(x86chunkList)}')
    print(f'x64 chunks: {len(x64chunkList)}')
    return x86chunkList, x64chunkList

x86chunkList, x64chunkList = dataGetter(dataThing, 1000, 1000)

dataset = BinaryClassificationDataset(x86chunkList, x64chunkList)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


In [88]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} for training")
model = BinaryClassifier().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()



Getting file info
Got file info


In [89]:
# initialize the data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


length of x86chunkList: 1000
size of x86chunkList[0]: 86
Extracted chunks
x86 chunks: 1000
x64 chunks: 1000
length of x86chunkList: 1000
size of x86chunkList[0]: 76
Extracted chunks
x86 chunks: 1000
x64 chunks: 1000


In [83]:
# train the model
num_epochs = 10
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        # forward pass
        scores = model(x)
        loss = criterion(scores, y)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        # gradient descent or adam step
        optimizer.step()
    print(f"Epoch {epoch} | Loss: {loss.item()}")


In [84]:
# save the model
torch.save(model.state_dict(), "model.ckpt")
print(f"Model saved to model.ckpt")

length of chunks: 1000
length of chunks[-1]: 20
datatype of first element of chunks: <class 'list'>


In [85]:
# test the model and generate ROC curve and conf mat
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

model = BinaryClassifier().to(device)
model.load_state_dict(torch.load("model.ckpt"))

print("Testing model")
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)
# the precedi

Using cuda:0 for training
